In [1]:
import os, treetaggerwrapper, networkx as nx, collections

import pandas as pd
import numpy as np
from HanTa import HanoverTagger as ht
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import GermanStemmer
from nltk.stem import PorterStemmer
from nltk import sent_tokenize, word_tokenize
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics, neighbors
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from pprint import pprint

H:\anaconda\lib\site-packages\treetaggerwrapper.py:739: FutureWarning: Possible nested set at position 8
  punct2find_re = re.compile("([^ ])([[" + ALONEMARKS + "])",
H:\anaconda\lib\site-packages\treetaggerwrapper.py:2043: FutureWarning: Possible nested set at position 152
  DnsHostMatch_re = re.compile("(" + DnsHost_expression + ")",
H:\anaconda\lib\site-packages\treetaggerwrapper.py:2067: FutureWarning: Possible nested set at position 409
  UrlMatch_re = re.compile(UrlMatch_expression, re.VERBOSE | re.IGNORECASE)
H:\anaconda\lib\site-packages\treetaggerwrapper.py:2079: FutureWarning: Possible nested set at position 192
  EmailMatch_re = re.compile(EmailMatch_expression, re.VERBOSE | re.IGNORECASE)


In [2]:
df = pd.read_csv('G:/Extractive-Summarisation-of-German-Wikipedia/dataset/processed.csv', encoding='utf-8')
df.head()

,Unnamed: 0,source,sim_sent,keywords,noCap_LetterWords_inSentence,no_words_inSent_SK,no_words_inSent_SS,sent_len
0,0,Minghella war der Sohn italienisch-schottische...,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",oscar regie opera bbc film bestes fernsehserie...,"[7, 6, 1, 4, 9, 9, 10, 8, 17, 37, 11, 7, 12, 7...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[2, 0, 0, 2, 1, 0, 2, 1, 3, 3, 0, 2, 0, 3, 2, ...","[17, 18, 6, 10, 17, 16, 28, 14, 31, 71, 17, 13..."
1,1,Ende der 1940er Jahre wurde eine erste Auteur-...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...",theorie film filme autor regisseur ansatz krit...,"[10, 7, 7, 6, 4, 3, 5, 5, 8, 5, 8, 13, 6, 6, 2...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[5, 4, 3, 3, 1, 1, 4, 6, 5, 2, 4, 5, 5, 7, 3, ...","[26, 17, 16, 13, 9, 8, 19, 15, 25, 19, 26, 36,..."
2,2,"Al Pacino, geboren in Manhattan, ist der Sohn ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",theatre al new the theaterstu richard yorker i...,"[15, 3, 13, 8, 7, 5, 17, 8, 4, 11, 3, 10, 11, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, ...","[8, 2, 6, 4, 2, 3, 8, 6, 4, 2, 3, 4, 5, 3, 11,...","[36, 11, 33, 16, 20, 12, 33, 18, 10, 21, 9, 19..."
3,3,Der Name der Alkalimetalle leitet sich von dem...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",sungen wasser eigenschaften reaktion lo erfolg...,"[8, 7, 2, 3, 5, 2, 2, 2, 6, 3, 6, 5, 3, 5, 7, ...","[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, ...","[4, 2, 2, 0, 2, 2, 2, 1, 2, 3, 4, 8, 3, 2, 4, ...","[21, 16, 5, 9, 13, 8, 6, 5, 18, 11, 16, 18, 9,..."
4,4,Die Arbeit ist bereits seit dem Altertum Gegen...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",geregelt ra gewerkschaften bgb betrieben arbei...,"[5, 5, 5, 11, 9, 8, 13, 8, 6, 4, 8, 5, 4, 10, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[2, 1, 0, 2, 1, 6, 2, 3, 3, 3, 3, 3, 1, 4, 4, ...","[10, 16, 9, 34, 18, 21, 35, 16, 18, 13, 22, 14..."


In [4]:
from sklearn.metrics.pairwise import cosine_similarity, linear_kernel
from nltk.corpus import stopwords

german_stop_words = stopwords.words('german')
def identify_tokens(row):
    tokens = sent_tokenize(row, language='german')
    return tokens

corpus = identify_tokens(df['source'][0])

#print(corpus)

tfidf_vectorizer = TfidfVectorizer(stop_words=german_stop_words)
tfidf_matrix = tfidf_vectorizer.fit_transform(corpus)

cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
linear_cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
print(cosine_sim)

[[1.         0.         0.         0.         0.02597783 0.02662211
  0.07371142 0.         0.01884577 0.01209965 0.         0.03562628
  0.04853386 0.02940111 0.11751224 0.06874723 0.02339968 0.03799937
  0.04485464 0.05478277 0.11043322 0.         0.26564482]
 [0.         1.         0.         0.1810938  0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.12421882 0.         0.         0.
  0.         0.         0.         0.         0.        ]
 [0.         0.         1.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.        ]
 [0.         0.1810938  0.         1.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.14517853 0.         0.         0.
  0.         0.         0.         0.         0.10554072]
 [0.02597783 0.         

In [6]:
import nltk, string, numpy
from nltk.corpus import stopwords
german_stop_words = stopwords.words('german')
nltk.download('punkt') # first-time use only
stemmer = nltk.stem.porter.PorterStemmer()
def StemTokens(tokens):
    return [stemmer.stem(token) for token in tokens]
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
def StemNormalize(text):
    return StemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

nltk.download('wordnet') # first-time use only
lemmer = nltk.stem.WordNetLemmatizer()
def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

from sklearn.feature_extraction.text import TfidfVectorizer
TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words=german_stop_words)
def cos_similarity(textlist):
    tfidf = TfidfVec.fit_transform(textlist)
    return (tfidf * tfidf.T).toarray()


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Dorian\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Dorian\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [9]:
def get_docs(df):
    threshold = 0.15
    doc = sent_tokenize(df, language="german")
    lst, tags, output_tuples, similar_sent = [], [], [], [] 
    arr = cos_similarity(doc)
    doc_names = ["s"+str(s) for s in range(0, len(doc), 1)]
    for row in range(len(arr)):
        lst = [row+1+idx for idx, num in \
                  enumerate(arr[row, row+1:]) if num >= threshold]
        [(output_tuples.append((doc[row], doc[item]))) for item in lst]
        print(arr)
'''    for tup in output_tuples:
        for sent in tup:
            similar_sent.append(sent)
    for sent in doc:
        if sent in set(doc).intersection(set(similar_sent)):
            tags.append(1)
        else:
            tags.append(0)
    return tags'''
#get_docs(df['source'][0])
#df['sim_sent'] = df['source'].apply(get_docs)

'    for tup in output_tuples:\n        for sent in tup:\n            similar_sent.append(sent)\n    for sent in doc:\n        if sent in set(doc).intersection(set(similar_sent)):\n            tags.append(1)\n        else:\n            tags.append(0)\n    return tags'

In [7]:
df.head(10)

,Unnamed: 0,source,sim_sent,keywords,noCap_LetterWords_inSentence,no_words_inSent_SK,no_words_inSent_SS,sent_len
0,0,Minghella war der Sohn italienisch-schottische...,"[1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, ...",oscar regie opera bbc film bestes fernsehserie...,"[7, 6, 1, 4, 9, 9, 10, 8, 17, 37, 11, 7, 12, 7...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[2, 0, 0, 2, 1, 0, 2, 1, 3, 3, 0, 2, 0, 3, 2, ...","[17, 18, 6, 10, 17, 16, 28, 14, 31, 71, 17, 13..."
1,1,Ende der 1940er Jahre wurde eine erste Auteur-...,"[0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, ...",theorie film filme autor regisseur ansatz krit...,"[10, 7, 7, 6, 4, 3, 5, 5, 8, 5, 8, 13, 6, 6, 2...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[5, 4, 3, 3, 1, 1, 4, 6, 5, 2, 4, 5, 5, 7, 3, ...","[26, 17, 16, 13, 9, 8, 19, 15, 25, 19, 26, 36,..."
2,2,"Al Pacino, geboren in Manhattan, ist der Sohn ...","[1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, ...",theatre al new the theaterstu richard yorker i...,"[15, 3, 13, 8, 7, 5, 17, 8, 4, 11, 3, 10, 11, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, ...","[8, 2, 6, 4, 2, 3, 8, 6, 4, 2, 3, 4, 5, 3, 11,...","[36, 11, 33, 16, 20, 12, 33, 18, 10, 21, 9, 19..."
3,3,Der Name der Alkalimetalle leitet sich von dem...,"[1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...",sungen wasser eigenschaften reaktion lo erfolg...,"[8, 7, 2, 3, 5, 2, 2, 2, 6, 3, 6, 5, 3, 5, 7, ...","[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, ...","[4, 2, 2, 0, 2, 2, 2, 1, 2, 3, 4, 8, 3, 2, 4, ...","[21, 16, 5, 9, 13, 8, 6, 5, 18, 11, 16, 18, 9,..."
4,4,Die Arbeit ist bereits seit dem Altertum Gegen...,"[1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",geregelt ra gewerkschaften bgb betrieben arbei...,"[5, 5, 5, 11, 9, 8, 13, 8, 6, 4, 8, 5, 4, 10, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[2, 1, 0, 2, 1, 6, 2, 3, 3, 3, 3, 3, 1, 4, 4, ...","[10, 16, 9, 34, 18, 21, 35, 16, 18, 13, 22, 14..."
5,5,"Mit ""Ampelkoalition"" wird in Deutschland übli...","[1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, ...",koalition fdp nen gru spd liberalen ndnis rot ...,"[18, 7, 7, 8, 3, 4, 5, 6, 19, 28, 7, 16, 22, 1...","[0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[7, 1, 6, 3, 2, 4, 2, 1, 4, 6, 3, 3, 3, 2, 2, ...","[29, 21, 26, 24, 10, 8, 12, 15, 36, 52, 17, 30..."
6,6,Im Mittelalter trat Ergotismus als Folge des V...,"[1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...",antonius getreide feuer krankheiten sekunda an...,"[10, 6, 7, 10, 3, 10, 3, 4, 9, 9, 10, 4, 3, 6,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[3, 2, 5, 1, 1, 5, 2, 1, 4, 2, 4, 1, 3, 1, 0, ...","[26, 20, 20, 22, 8, 25, 12, 9, 20, 14, 22, 8, ..."
7,7,Arnims Vater war der wohlhabende Königlich Pr...,"[0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, ...",goethe erza weimar go grimm berlin berliner sc...,"[18, 5, 3, 13, 5, 9, 9, 8, 4, 9, 6, 5, 8, 7, 1...","[0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...","[4, 1, 0, 4, 2, 3, 1, 6, 2, 1, 2, 3, 3, 2, 3, ...","[36, 7, 7, 32, 14, 24, 17, 18, 14, 17, 17, 17,..."
8,8,Es gibt drei klassische Aggregatzustände: Fü...,"[1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, ...",temperatur flu gas phase bewegung ssigkeit sto...,"[7, 9, 6, 4, 5, 5, 4, 4, 4, 4, 2, 7, 3, 3, 10,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[6, 10, 4, 6, 7, 4, 1, 3, 1, 5, 0, 2, 1, 0, 3,...","[18, 28, 15, 12, 19, 14, 9, 11, 14, 19, 2, 19,..."
9,9,"Börsenkurse sind dadurch gekennzeichnet, dass...","[0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, ...",bo handels handel rse kurs nnen ftsfu aktien d...,"[2, 5, 7, 3, 4, 13, 9, 5, 7, 8, 7, 3, 15, 6, 7...","[1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 3, 4, 0, 1, 8, 5, 1, 1, 3, 6, 2, 6, 1, 7, ...","[12, 16, 28, 7, 7, 34, 34, 13, 16, 18, 25, 15,..."


In [8]:
import pandas as pd, re, nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

german_stop_words = stopwords.words('german')

cv = CountVectorizer(max_df=0.85,stop_words=german_stop_words, max_features=10000)
word_count_vector = cv.fit_transform(df['source'])

In [9]:
tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(word_count_vector)

TfidfTransformer()

In [10]:
tfidf_transformer.idf_
feature_names=cv.get_feature_names()

In [11]:
def sort_coo(coo_matrix):
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)

def extract_topn_from_vector(feature_names, sorted_items, topn=10):
    """get the feature names and tf-idf score of top n items"""
    
    #use only topn items from vector
    sorted_items = sorted_items[:topn]

    score_vals = []
    feature_vals = []

    for idx, score in sorted_items:
        fname = feature_names[idx]
        
        #keep track of feature name and its corresponding score
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])

    #create a tuples of feature,score
    #results = zip(feature_vals,score_vals)
    results= {}
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]]=score_vals[idx]
    
    return results

In [12]:
def keyw(df):
    kljuc = []
    tf_idf_vector=tfidf_transformer.transform(cv.transform([df]))
    sorted_items=sort_coo(tf_idf_vector.tocoo())
    keywords = extract_topn_from_vector(feature_names,sorted_items,10)
    for k in keywords:
        kljuc.append(k)
    return kljuc

df['keywords'] = df['source'].apply(keyw)

In [13]:
df.head()

,Unnamed: 0,source,sim_sent,keywords,noCap_LetterWords_inSentence,no_words_inSent_SK,no_words_inSent_SS,sent_len
0,0,Minghella war der Sohn italienisch-schottische...,"[1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, ...","[oscar, regie, opera, bbc, film, bestes, ferns...","[7, 6, 1, 4, 9, 9, 10, 8, 17, 37, 11, 7, 12, 7...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[2, 0, 0, 2, 1, 0, 2, 1, 3, 3, 0, 2, 0, 3, 2, ...","[17, 18, 6, 10, 17, 16, 28, 14, 31, 71, 17, 13..."
1,1,Ende der 1940er Jahre wurde eine erste Auteur-...,"[0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, ...","[theorie, film, filme, autor, regisseur, ansat...","[10, 7, 7, 6, 4, 3, 5, 5, 8, 5, 8, 13, 6, 6, 2...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[5, 4, 3, 3, 1, 1, 4, 6, 5, 2, 4, 5, 5, 7, 3, ...","[26, 17, 16, 13, 9, 8, 19, 15, 25, 19, 26, 36,..."
2,2,"Al Pacino, geboren in Manhattan, ist der Sohn ...","[1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, ...","[theatre, al, new, the, theaterstu, richard, y...","[15, 3, 13, 8, 7, 5, 17, 8, 4, 11, 3, 10, 11, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, ...","[8, 2, 6, 4, 2, 3, 8, 6, 4, 2, 3, 4, 5, 3, 11,...","[36, 11, 33, 16, 20, 12, 33, 18, 10, 21, 9, 19..."
3,3,Der Name der Alkalimetalle leitet sich von dem...,"[1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...","[sungen, wasser, eigenschaften, reaktion, lo, ...","[8, 7, 2, 3, 5, 2, 2, 2, 6, 3, 6, 5, 3, 5, 7, ...","[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, ...","[4, 2, 2, 0, 2, 2, 2, 1, 2, 3, 4, 8, 3, 2, 4, ...","[21, 16, 5, 9, 13, 8, 6, 5, 18, 11, 16, 18, 9,..."
4,4,Die Arbeit ist bereits seit dem Altertum Gegen...,"[1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[geregelt, ra, gewerkschaften, bgb, betrieben,...","[5, 5, 5, 11, 9, 8, 13, 8, 6, 4, 8, 5, 4, 10, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[2, 1, 0, 2, 1, 6, 2, 3, 3, 3, 3, 3, 1, 4, 4, ...","[10, 16, 9, 34, 18, 21, 35, 16, 18, 13, 22, 14..."


In [15]:
def capital_lWords(df):
    tokens = sent_tokenize(df, language="german")
    return [sum([c.isupper() for c in a]) for a in tokens]

df['noCap_LetterWords_inSentence'] = df['source'].apply(capital_lWords)
df.head()

,Unnamed: 0,source,sim_sent,keywords,noCap_LetterWords_inSentence,no_words_inSent_SK,no_words_inSent_SS,sent_len
0,0,Minghella war der Sohn italienisch-schottische...,"[1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, ...","[oscar, regie, opera, bbc, film, bestes, ferns...","[7, 6, 1, 4, 9, 9, 10, 8, 17, 37, 11, 7, 12, 7...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[2, 0, 0, 2, 1, 0, 2, 1, 3, 3, 0, 2, 0, 3, 2, ...","[17, 18, 6, 10, 17, 16, 28, 14, 31, 71, 17, 13..."
1,1,Ende der 1940er Jahre wurde eine erste Auteur-...,"[0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, ...","[theorie, film, filme, autor, regisseur, ansat...","[10, 7, 7, 6, 4, 3, 5, 5, 8, 5, 8, 13, 6, 6, 2...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[5, 4, 3, 3, 1, 1, 4, 6, 5, 2, 4, 5, 5, 7, 3, ...","[26, 17, 16, 13, 9, 8, 19, 15, 25, 19, 26, 36,..."
2,2,"Al Pacino, geboren in Manhattan, ist der Sohn ...","[1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, ...","[theatre, al, new, the, theaterstu, richard, y...","[15, 3, 13, 8, 7, 5, 17, 8, 4, 11, 3, 10, 11, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, ...","[8, 2, 6, 4, 2, 3, 8, 6, 4, 2, 3, 4, 5, 3, 11,...","[36, 11, 33, 16, 20, 12, 33, 18, 10, 21, 9, 19..."
3,3,Der Name der Alkalimetalle leitet sich von dem...,"[1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...","[sungen, wasser, eigenschaften, reaktion, lo, ...","[8, 7, 2, 3, 5, 2, 2, 2, 6, 3, 6, 5, 3, 5, 7, ...","[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, ...","[4, 2, 2, 0, 2, 2, 2, 1, 2, 3, 4, 8, 3, 2, 4, ...","[21, 16, 5, 9, 13, 8, 6, 5, 18, 11, 16, 18, 9,..."
4,4,Die Arbeit ist bereits seit dem Altertum Gegen...,"[1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[geregelt, ra, gewerkschaften, bgb, betrieben,...","[5, 5, 5, 11, 9, 8, 13, 8, 6, 4, 8, 5, 4, 10, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[2, 1, 0, 2, 1, 6, 2, 3, 3, 3, 3, 3, 1, 4, 4, ...","[10, 16, 9, 34, 18, 21, 35, 16, 18, 13, 22, 14..."


In [63]:
def W_sourceSummary(df, df2):
    f = []
    text2 = df2
    text2 = text2.translate(str.maketrans('', '', string.punctuation))
    tokens = sent_tokenize(df, language = "german")
    tok2 = word_tokenize(text2, language = "german")
    for s in tokens:
        f.append(len(list((set(s.split()).intersection(set(tok2))))))
    return f

In [64]:
def W_sourceKeywords(df, df2):
    f = []
    df.lower()
    text2 = df2
    text2 = text2.translate(str.maketrans('', '', string.punctuation))
    tokens = sent_tokenize(df, language = "german")
    tok2 = word_tokenize(text2, language = "german")
    for s in tokens:
        f.append(len(list((set(s.split()).intersection(set(tok2))))))
    return f

def listToString(df):
    string = ' '.join([str(e) for e in df])
    return string

In [65]:
df['keywords'] = df['keywords'].apply(listToString)
df['no_words_inSent_SK'] = df.apply(lambda x: W_sourceKeywords(x['source'], x['keywords']), axis=1)

In [66]:
df['no_words_inSent_SS'] = df.apply(lambda x: W_sourceSummary(x['source'], x['summary']), axis=1)

In [67]:
lst = []
tokens = sent_tokenize(df['source'][0], language='german')
for s in tokens:
    words = s.split()
    lst.append(words)
print(lst)

[['Minghella', 'war', 'der', 'Sohn', 'italienisch-schottischer', 'Eltern,', 'die', 'auf', 'der', 'Isle', 'of', 'Wight', 'eine', 'Fabrik', 'für', 'Eiscreme', 'betrieben.'], ['Nach', 'seinem', 'Schulabschluss', 'studierte', 'er', 'an', 'der', 'Universität', 'Hull,', 'wo', 'er', 'eine', 'Zeit', 'lang', 'als', 'Dozent', 'tätig', 'war.'], ['1978', 'drehte', 'er', 'einen', 'ersten', 'Kurzfilm.'], ['Seit', '1981', 'war', 'er', 'als', 'Autor', 'und', 'Story', 'Editor', 'tätig.'], ['Er', 'wurde', 'mit', 'Theaterstücken,', 'Rundfunkhörspielen,', 'der', 'Fernsehserie', '"Inspector', 'Morse"', 'und', 'vielen', 'Drehbüchern', 'für', 'Film', 'und', 'Fernsehen', 'bekannt.'], ['Er', 'entwickelte', 'die', 'Drehbücher', 'für', 'die', '1988', 'erfolgreich', 'ausgestrahlte', 'Fernsehserie', 'The', 'Storyteller', 'von', 'Muppets-Erfinder', 'Jim', 'Henson.'], ['Auch', 'als', 'Produzent', 'war', 'er', 'erfolgreich,', 'darunter', 'für', 'die', 'Filme', '"Der', 'stille', 'Amerikaner",', '"Die', 'Dol

In [68]:
def s_len(row):
    lst  = []
    tokens = sent_tokenize(row, language='german')
    [(lst.append(len(s.split()))) for s in tokens]
    return lst
df['sent_len'] = df['source'].apply(s_len)

In [69]:
#df.to_csv('G:/Extractive-Summarisation-of-German-Wikipedia/dataset/processed.csv')
df.head()

,source,summary,sim_sent,keywords,noCap_LetterWords_inSentence,no_words_inSent_SK,no_words_inSent_SS,sent_len
0,Minghella war der Sohn italienisch-schottische...,"Anthony Minghella, CBE war ein britischer Film...","[1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, ...",oscar regie opera bbc film bestes fernsehserie...,"[7, 6, 1, 4, 9, 9, 10, 8, 17, 37, 11, 7, 12, 7...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[2, 0, 0, 2, 1, 0, 2, 1, 3, 3, 0, 2, 0, 3, 2, ...","[17, 18, 6, 10, 17, 16, 28, 14, 31, 71, 17, 13..."
1,Ende der 1940er Jahre wurde eine erste Auteur-...,Die Auteur-Theorie ist eine Filmtheorie und di...,"[0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, ...",theorie film filme autor regisseur ansatz krit...,"[10, 7, 7, 6, 4, 3, 5, 5, 8, 5, 8, 13, 6, 6, 2...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[5, 4, 3, 3, 1, 1, 4, 6, 5, 2, 4, 5, 5, 7, 3, ...","[26, 17, 16, 13, 9, 8, 19, 15, 25, 19, 26, 36,..."
2,"Al Pacino, geboren in Manhattan, ist der Sohn ...","Alfredo James ""Al"" Pacino ist ein US-amerikani...","[1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, ...",theatre al new the theaterstu richard yorker i...,"[15, 3, 13, 8, 7, 5, 17, 8, 4, 11, 3, 10, 11, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, ...","[8, 2, 6, 4, 2, 3, 8, 6, 4, 2, 3, 4, 5, 3, 11,...","[36, 11, 33, 16, 20, 12, 33, 18, 10, 21, 9, 19..."
3,Der Name der Alkalimetalle leitet sich von dem...,Als Alkalimetalle werden die chemischen Elemen...,"[1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...",sungen wasser eigenschaften reaktion lo erfolg...,"[8, 7, 2, 3, 5, 2, 2, 2, 6, 3, 6, 5, 3, 5, 7, ...","[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, ...","[4, 2, 2, 0, 2, 2, 2, 1, 2, 3, 4, 8, 3, 2, 4, ...","[21, 16, 5, 9, 13, 8, 6, 5, 18, 11, 16, 18, 9,..."
4,Die Arbeit ist bereits seit dem Altertum Gegen...,Das deutsche Arbeitsrecht ist ein Rechtsgebiet...,"[1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",geregelt ra gewerkschaften bgb betrieben arbei...,"[5, 5, 5, 11, 9, 8, 13, 8, 6, 4, 8, 5, 4, 10, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[2, 1, 0, 2, 1, 6, 2, 3, 3, 3, 3, 3, 1, 4, 4, ...","[10, 16, 9, 34, 18, 21, 35, 16, 18, 13, 22, 14..."


In [74]:
df['source'] = df.apply(lambda x: sent_tokenize(x['source'], language="german"), axis=1)

In [76]:
print (df[0:5])

                                              source  \
0  [Minghella war der Sohn italienisch-schottisch...   
1  [Ende der 1940er Jahre wurde eine erste Auteur...   
2  [Al Pacino, geboren in Manhattan, ist der Sohn...   
3  [Der Name der Alkalimetalle leitet sich von de...   
4  [Die Arbeit ist bereits seit dem Altertum Gege...   

                                            sim_sent  \
0  [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   
1  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...   
2  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   
3  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   
4  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   

                                            keywords  \
0  oscar regie opera bbc film bestes fernsehserie...   
1  theorie film filme autor regisseur ansatz krit...   
2  theatre al new the theaterstu richard yorker i...   
3  sungen wasser eigenschaften reaktion lo erfolg...   
4  geregelt ra gewerkschaften bgb betrieben ar

In [43]:
german_stop_words = stopwords.words('german')
def identify_tokens(row):
    tokens = sent_tokenize(row, language='german')
    tfidf_vectorizer = TfidfVectorizer(stop_words=german_stop_words, max_df=0.8)
    tfidf_matrix = tfidf_vectorizer.fit_transform(tokens).toarray()
    return tfidf_matrix

#df['source'] = df['source'].apply(identify_tokens)

#print(tfidf_matrix)

df2 = df.head(1)
#corpus = sent_tokenize(df2['source'][0], language="german")
#tfidf_vectorizer = TfidfVectorizer(stop_words=german_stop_words, min_df=0.1, max_df=0.8)
#tfidf_matrix = tfidf_vectorizer.fit_transform(corpus).toarray()
#print(tfidf_matrix)
df['source'] = df['source'].apply(identify_tokens)

In [40]:
df2.head()

,Unnamed: 0,source,sim_sent,keywords,noCap_LetterWords_inSentence,no_words_inSent_SK,no_words_inSent_SS,sent_len
0,0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",oscar regie opera bbc film bestes fernsehserie...,"[7, 6, 1, 4, 9, 9, 10, 8, 17, 37, 11, 7, 12, 7...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[2, 0, 0, 2, 1, 0, 2, 1, 3, 3, 0, 2, 0, 3, 2, ...","[17, 18, 6, 10, 17, 16, 28, 14, 31, 71, 17, 13..."


In [52]:
import re
from sklearn.model_selection import train_test_split
X = df[['source', 'noCap_LetterWords_inSentence', 'no_words_inSent_SK', 'no_words_inSent_SS', 'sent_len']]
y = df['sim_sent']

from nltk.stem import WordNetLemmatizer

documents = []

stemmer = WordNetLemmatizer()

for sen in range(0, len(X)):
    # Remove all the special characters
    document = re.sub(r'\W', ' ', str(X[sen]))
    
    # remove all single characters
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
    
    # Remove single characters from the start
    document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 
    
    # Substituting multiple spaces with single space
    document = re.sub(r'\s+', ' ', document, flags=re.I)
    
    # Removing prefixed 'b'
    document = re.sub(r'^b\s+', '', document)
    
    # Converting to Lowercase
    document = document.lower()
    
    # Lemmatization
    document = document.split()

    document = [stemmer.lemmatize(word) for word in document]
    document = ' '.join(document)
    
    documents.append(document)

from sklearn.feature_extraction.text import TfidfTransformer
tfidfconverter = TfidfTransformer()
X = tfidfconverter.fit_transform(X).toarray()

KeyError: 0

In [45]:
from sklearn.ensemble import RandomForestClassifier

clf=RandomForestClassifier(n_estimators=100)
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)

ValueError: setting an array element with a sequence.

In [ ]:
from sklearn import metrics
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))